In [ ]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.1 MB/s eta 0:00:00


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import openai
from sklearn.decomposition import PCA
import numpy as np

# 벡터 데이터 예시, 실제 데이터로 대체해야 합니다.
# 예를 들어, embedding['data'][0]['embedding'] 가 벡터 데이터를 포함하고 있다면,
# 아래 embeddings에 해당 데이터를 넣어야 합니다.

# 발급받은 API 키 설정
OPENAI_API_KEY = ""

# openai API 키 인증
openai.api_key = OPENAI_API_KEY
pc = Pinecone(api_key="")

# Pinecone 인덱스 초기화
index = pc.Index("yoo-img")

file_path = "/content/yoo_img.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    data = file.readlines()

# 각 텍스트 라인에 대한 임베딩 생성
embeddings = openai.Embedding.create(
    model="text-embedding-3-small",
    input=data
)

# 각 텍스트 라인의 ID와 메타데이터 생성
ids = [str(x) for x in range(len(data))]
meta_datas = [{'text': text.strip()} for text in data]

# (id, vector, metadata) 형식의 레코드 준비
records = [(id, emb['embedding'], meta) for id, emb, meta in zip(ids, embeddings['data'], meta_datas)]

# Pinecone 인덱스에 벡터 업로드
index.upsert(vectors=records)

{'upserted_count': 74}

In [ ]:
question = "웃는 얼굴"

embedding = openai.Embedding.create(
    model ="text-embedding-3-small",
    input = question
)
# 원래의 벡터
embadding_vector = embedding['data'][0]['embedding']

query_result=index.query(
  vector=embadding_vector,
  top_k=5,
  include_values=False,
  include_metadata=True
)

result_ids = [ result.id for result in query_result.matches]

for result in query_result.matches:
  id = result.id
  text = result.metadata['text'] #문서의 원본 텍스트
  score = result.score #문서의 유사도
  print(id,score,text)

43 0.560672283 웃고있는_멀리있는_아래를보는_옆모습.png
50 0.514038503 웃고있는_위에서보는_반팔_실내.png
57 0.511557519 웃고있는_화사한_셀카_정면.png
51 0.508919537 웃고있는_인자한_어색한_양복_정면.png
49 0.504846156 웃고있는_웃긴_못생긴_패딩_정면.png
